# [awesome page on pivot table](https://pbpython.com/pandas-pivot-table-explained.html)

In [2]:
import pandas as pd
import numpy as np

users = pd.read_csv("data/movielens/users.dat", header=None, sep='::')
ratings = pd.read_csv("data/movielens/ratings.dat", header=None, sep='::')
movies = pd.read_csv("data/movielens/movies.dat", header=None, sep='::')

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can

In [4]:
# column names are in the readme file. bring em. smash em, stick em on a sheet

ratings_h = "UserID::MovieID::Rating::Timestamp"
users_h = 'UserID::Gender::Age::Occupation::Zip-code'
movies_h = 'MovieID::Title::Genres'

users.columns = users_h.split('::')
ratings.columns = ratings_h.split('::')
movies.columns = movies_h.split('::')

In [10]:
df = pd.merge(pd.merge(movies, ratings), users)
print(df.head(5))

# to get ratings by gender
df_fm = pd.pivot_table(df, index = ['MovieID','Title'], columns='Gender',values='Rating', aggfunc='mean')
print(df_fm.head(5))

   MovieID                                      Title  \
0        1                           Toy Story (1995)   
1       48                          Pocahontas (1995)   
2      150                           Apollo 13 (1995)   
3      260  Star Wars: Episode IV - A New Hope (1977)   
4      527                    Schindler's List (1993)   

                                 Genres  UserID  Rating  Timestamp Gender  \
0           Animation|Children's|Comedy       1       5  978824268      F   
1  Animation|Children's|Musical|Romance       1       5  978824351      F   
2                                 Drama       1       5  978301777      F   
3       Action|Adventure|Fantasy|Sci-Fi       1       4  978300760      F   
4                             Drama|War       1       5  978824195      F   

   Age  Occupation Zip-code  
0    1          10    48067  
1    1          10    48067  
2    1          10    48067  
3    1          10    48067  
4    1          10    48067  
Gender        

In [11]:
# but I also want to know how many ratings were made
f = df_fm_counts = pd.pivot_table(df, index = ['MovieID','Title'], columns='Gender',values='Rating', aggfunc='count')
f.columns.get_level_values(0).unique()
f.columns=['F_count','M_count']

#pd.merge(df_fm,f) #not merge since it is joining on index
df_fmc = df_fm.join(f, how='outer')


df_fmc['diff'] = np.abs(df_fmc['F'] - df_fmc['M'])
df_fmc.sort_index(by='diff', ascending=False).head(5)
#this is garbage

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  # This is added back by InteractiveShellApp.init_path()


,,F,M,F_count,M_count,diff
MovieID,Title,,,,,
682,Tigrero: A Film That Was Never Made (1994),1.0,4.333333,1.0,3.0,3.333333
687,Country Life (1994),5.0,2.000000,1.0,2.0,3.000000
3136,"James Dean Story, The (1957)",4.0,1.000000,2.0,1.0,3.000000
2823,"Spiders, The (Die Spinnen, 1. Teil: Der Goldene See) (1919)",4.0,1.000000,2.0,2.0,3.000000
138,"Neon Bible, The (1995)",1.0,4.000000,1.0,1.0,3.000000


In [0]:
# taking movies that got more than 50 ratings from female, at least
df_fm[df_fm['F_count']>=50].sort_index(by='diff', ascending=False).head(10)

# so there is not much gender difference on movie ratings, which kinda makes sense

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


,,F,M,F_count,M_count,diff
MovieID,Title,,,,,
1088,Dirty Dancing (1987),3.790378,2.959596,291.0,396.0,0.830782
1201,"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,99.0,723.0,0.726351
203,"To Wong Foo, Thanks for Everything! Julie Newmar (1995)",3.486842,2.795276,76.0,127.0,0.691567
2468,Jumpin' Jack Flash (1986),3.254717,2.578358,106.0,268.0,0.676359
506,Orlando (1993),3.862745,3.190476,51.0,63.0,0.672269
613,Jane Eyre (1996),3.839286,3.192308,56.0,26.0,0.646978
231,Dumb & Dumber (1994),2.697987,3.336595,149.0,511.0,0.638608
784,"Cable Guy, The (1996)",2.250000,2.863787,84.0,301.0,0.613787
1261,Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,74.0,474.0,0.611985


In [14]:
#reviewing regex a bit

import re


df2 = df.copy()


def getyear(title):
  pat = re.compile('[(]\d*[)]')
  year = pat.findall(title)[0]
  #need year.islist() kind of line
  return year

def gettitle(title):
  title = title.split('(')[0]
  return title

df2['years'] = df2['Title'].apply(getyear)
df2['Title'] = df2['Title'].apply(gettitle)

df2.head(4)

# b = pat.search('3 rings (for) eleven kings') # searches until the first match
# #c = pat.findall('they are taking the hobbits to isengard')
# print(b)

,MovieID,Title,Genres,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,years
0,1,Toy Story,Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,False
1,48,Pocahontas,Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,False
2,150,Apollo 13,Drama,1,5,978301777,F,1,10,48067,False
3,260,Star Wars: Episode IV - A New Hope,Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,False


# one hot encoding!

In [17]:
df3 = df.copy()

G_list = []
for i in range(df3.shape[0]):
  a = df3['Genres'][i].split('|')
  for j in a:
     if j not in G_list:
        G_list.append(j)

print (G_list)

# scratch this approach

['Animation', "Children's", 'Comedy', 'Musical', 'Romance', 'Drama', 'Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'War', 'Crime', 'Thriller', 'Western', 'Horror', 'Mystery', 'Documentary', 'Film-Noir']


In [18]:
df4 = df3.loc[:,['Title', 'Genres']]

def divi(combi):
  return combi.split('|')

df4['Genres']=df4['Genres'].apply(divi)

df4.head(4)


,Title,Genres
0,Toy Story (1995),"[Animation, Children's, Comedy]"
1,Pocahontas (1995),"[Animation, Children's, Musical, Romance]"
2,Apollo 13 (1995),[Drama]
3,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Fantasy, Sci-Fi]"


In [19]:
from sklearn.preprocessing import MultiLabelBinarizer


df4 = df3.loc[:,['Title', 'Genres']]

def divi(combi):
  return combi.split('|')

df4['Genres']=df4['Genres'].apply(divi)


mlb = MultiLabelBinarizer()
df4 = df4.join(pd.DataFrame(mlb.fit_transform(df4.pop('Genres')),
                          columns=mlb.classes_, index=df4.index))
df4

,Title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Pocahontas (1995),0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
2,Apollo 13 (1995),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Star Wars: Episode IV - A New Hope (1977),1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,Schindler's List (1993),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,"Secret Garden, The (1993)",0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6,Aladdin (1992),0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
7,Snow White and the Seven Dwarfs (1937),0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,Beauty and the Beast (1991),0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,Fargo (1996),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
